# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,74.16,81,0,10.58,MG,1732331318
1,1,naze,28.3667,129.4833,66.81,56,3,12.64,JP,1732331319
2,2,ayr,55.4627,-4.6339,35.58,94,100,13.80,GB,1732331285
3,3,al ghayzah,16.2079,52.1760,74.62,76,83,5.01,YE,1732331321
4,4,utrik,11.2278,169.8474,82.96,79,36,20.15,MH,1732331323


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
humidityplot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# max temp: 27 C ~ 80.6 F
# min temp: 21 C ~ 69.8 F
# wind < 4.5
# cloudiness = 0

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
idealCityDF = idealCityDF.dropna()


# Display sample data
print(len(idealCityDF))
idealCityDF.head()

6


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
117,117,tura,25.5198,90.2201,70.45,68,0,1.77,IN,1732331464
149,149,tsiombe,-25.3000,45.4833,77.88,68,0,3.69,MG,1732331235
198,198,ixtapa,20.7000,-105.2000,78.22,89,0,3.44,MX,1732331563
273,273,remire-montjoly,4.9167,-52.2667,77.04,94,0,0.00,GF,1732331663
469,469,saint-pierre,-21.3393,55.4781,78.48,83,0,3.44,RE,1732331904


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
117,tura,IN,25.5198,90.2201,68,
149,tsiombe,MG,-25.3000,45.4833,68,
198,ixtapa,MX,20.7000,-105.2000,89,
273,remire-montjoly,GF,4.9167,-52.2667,94,
469,saint-pierre,RE,-21.3393,55.4781,83,
495,tilisarao,AR,-32.7329,-65.2911,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apikey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
tilisarao - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE